In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import zipfile
import os
import re
from collections import Counter
from bs4 import BeautifulSoup

In [ ]:
df = pd.read_csv("combined_data.csv")

In [ ]:
df.drop(columns=["zarobki", "Unnamed: 0"])

In [ ]:
df.drop(df[df.zarobki_min == "Brak danych"].index, inplace=True)
df


In [ ]:
df[df["nazwa"].str.contains("Python")]

In [ ]:
df[df.miasto == "Poznań"]

In [ ]:
def read_shape_from_zip(zip_filename, entry_name_regex):
    z = zipfile.ZipFile(zip_filename, mode="r")
    dbf_name = list(filter(
        lambda x: re.match(entry_name_regex + r'.*\.dbf$',
                           os.path.basename(x)),
        z.namelist()
    ))[0]
    z.close()
    return geopandas.read_file(dbf_name, encoding='utf-8', vfs='zip://'+zip_filename)

In [ ]:
wojewodztwa = read_shape_from_zip("Wojewodztwa.zip", "Województwa")

In [ ]:
wojewodztwa.plot()

In [ ]:
lista_miast = []
for miasta in df.miasto.to_list():
    if ("+" in miasta):
        lista_miast.append(miasta.split("+")[0].split(",")[0].strip().upper())
    elif "," in miasta:
        for miasto in miasta.split(","):
            lista_miast.append(miasto.strip().upper())
    elif (miasta == "Nie podano"):
        pass

    else:
        lista_miast.append(miasta.upper())

len(lista_miast)

In [ ]:
for miasto in lista_miast:
    if "SUCHY LAS" in miasto:
        lista_miast.remove(miasto)
        lista_miast.append("Poznań".upper())
    if "WARSAW" in miasto:
        lista_miast.remove(miasto)
        lista_miast.append("WARSZAWA")

In [ ]:
zbior_miast = set(lista_miast)

In [ ]:
len(zbior_miast)

In [ ]:
zbior_miast

In [ ]:
ilosc_ofert_na_miasto = Counter(lista_miast)

In [ ]:
ilosc_ofert_na_miasto

In [ ]:
with open("miasta.html") as miasta:
    soup = BeautifulSoup(miasta, "html.parser")

role = soup.find_all("tr", {"role":"row"})

lista_miast = []
for i in range(1, len(role)):
    dict = {"miasto" : role[i].find_all("a")[0].text.strip().upper(), "wojewodztwo" : role[i].find_all("a")[2].text.strip().upper()}
    lista_miast.append(dict)

df = pd.DataFrame(lista_miast)
df['ilosc'] = 0
df

In [ ]:
df.loc[df['miasto'] == 'WARSZAWA', 'ilosc'] = 10

In [ ]:
for miasto in ilosc_ofert_na_miasto.keys():
    df.loc[df['miasto'] == miasto, 'ilosc'] = ilosc_ofert_na_miasto.get(miasto)

In [ ]:
df

In [ ]:
df = df.drop(df[df.ilosc == 0].index)

In [ ]:
df

In [ ]:
wojewodztwa['JPT_NAZWA_'] = wojewodztwa['JPT_NAZWA_'].str.upper()
wojewodztwa.rename(columns={"JPT_NAZWA_" : "wojewodztwo"}, inplace=True)
wojewodztwa

In [ ]:
wojewodztwa_ilosc = df.drop(columns="miasto")
wojewodztwa_ilosc = wojewodztwa_ilosc.groupby('wojewodztwo').sum()
wojewodztwa_ilosc

df_wojewodztwa_ilosc = pd.merge(wojewodztwa, wojewodztwa_ilosc, left_on='wojewodztwo', right_index=True)
df_wojewodztwa_ilosc.sort_values('ilosc',ascending=False)
# df_wojewodztwa_ilosc

# df_district_qnty = pd.merge(districts, df_district, left_on='nazwa', right_index=True)
# df_district_qnty.sort_values('ilosc',ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
df_wojewodztwa_ilosc.plot(column='ilosc', cmap='spring_r', linewidth=1.5, ax=ax)

vmin, vmax = df_wojewodztwa_ilosc['ilosc'].min(), df_wojewodztwa_ilosc['ilosc'].max()
sm = plt.cm.ScalarMappable(cmap='spring_r', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
fig.colorbar(sm, shrink=0.5, ax=ax)

ax.grid(True)
ax.set_axis_off()
ax.set_title("Liczba ofert ze względu na województwo")
plt.tight_layout()
plt.show()

In [ ]:
df = pd.read_csv("combined_data_with_average_salary.csv")
df = df.drop(columns=["Unnamed: 0"])

In [ ]:
df

In [ ]:
lista_miast2 = []

for i in range(0, len(df)):
    miasta = df.iloc[i].get('miasto')
    zarobki_srednie = df.iloc[i].get('zarobki_srednie')

    if ("+" in miasta):
        lista_miast2.append(miasta.split("+")[0].split(",")[0].strip().upper())
        lista_miast2.append(zarobki_srednie)
    elif "," in miasta:
        for miasto in miasta.split(","):
            lista_miast2.append(miasto.strip().upper())
            lista_miast2.append(zarobki_srednie)
    elif (miasta == "Nie podano"):
        pass

    else:
        lista_miast2.append(miasta.upper())
        lista_miast2.append(zarobki_srednie)

len(lista_miast2)

In [ ]:
lista_miast2

In [ ]:
d = {}

for miasto, ilosc in ilosc_ofert_na_miasto.items():
    d[miasto] = ilosc

import numpy

In [ ]:
d2 = {}

for miasto, ilosc in ilosc_ofert_na_miasto.items():
    d2[miasto] = 0.0

for i in range(0, 14140):
    if(i % 2 == 0):
        if d.get(lista_miast2[i]) is not None:
            if numpy.isnan(lista_miast2[i+1]) == False:
                d2[lista_miast2[i]] += lista_miast2[i+1]   

print(len(d2))
print(d2)

In [397]:
d3 = {}
import math

for miasto, ilosc in ilosc_ofert_na_miasto.items():
    d3[miasto] = 0.0

for miasto in zbior_miast:
    d3[miasto] = math.ceil(d2.get(miasto) / d.get(miasto))

In [398]:
d3

{'WARSZAWA': 17487,
 'GDAŃSK': 16222,
 'WROCŁAW': 16787,
 'KRAKÓW': 18089,
 'KATOWICE': 15205,
 'ŁÓDŹ': 15590,
 'OPOLE': 9358,
 'KIELCE': 20665,
 'KRAKOW': 16820,
 'RZESZÓW': 14947,
 'GDYNIA': 13751,
 'BIAŁYSTOK': 18117,
 'LUBLIN': 17242,
 'SZTOKHOLM': 13500,
 'GŻIRA': 18217,
 'POZNAŃ': 14987,
 'TBILISI': 18217,
 'ZIELONA GÓRA': 19980,
 'BYDGOSZCZ': 16522,
 'CZĘSTOCHOWA': 16019,
 'GLIWICE': 15438,
 'BERLIN': 26479,
 'KEMPTEN (ALLGÄU)': 19100,
 'PARYŻ': 19753,
 'ULM': 15875,
 'SZCZECIN': 15520,
 'BIELSKO-BIAŁA': 13723,
 'PŁOCK': 15567,
 'SOSNOWIEC': 22500,
 'PIŁA': 22900,
 'OSTRÓW WIELKOPOLSKI': 12567,
 'ATENY': 11000,
 'LIMASSOL': 14388,
 'ANIF': 11000,
 'SALZBURG': 11000,
 'PABIANICE': 11000,
 'PIEŃKÓW': 11000,
 'ZABIERZÓW': 13650,
 'SZCZAWNO-ZDRÓJ': 11000,
 'STAROGARD GDAŃSKI': 11000,
 'TORUŃ': 16138,
 'JELCZ LASKOWICE': 11000,
 'WAŁBRZYCH': 11000,
 'STRYKÓW': 11000,
 'LONDON': 28949,
 'KALISZ': 8250,
 'AUSTIN': 32380,
 'IXELLES': 24390,
 'ESCH-SUR-ALZETTE': 26710,
 'MEDIOLAN': 15330